In [25]:
import xarray as xr
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import os
import numpy as np

from scipy.spatial.distance import cdist

In [ ]:
tasmax_file = "data/REA_ANASTASIA_raw/tasmax_France-Metro_ANASTASIA-V1_day_20190101_20191231.nc"
tasmax_ds = xr.open_dataset(f"../{tasmax_file}")

tasmax_ds

<xarray.Dataset> Size: 2GB
Dimensions:         (y: 1081, x: 1171, nbounds: 4, time: 365)
Coordinates:
  * y               (y) float64 9kB 1.605e+06 1.606e+06 ... 2.684e+06 2.685e+06
  * x               (x) float64 9kB 4e+04 4.1e+04 4.2e+04 ... 1.209e+06 1.21e+06
  * time            (time) datetime64[ns] 3kB 2019-01-01 ... 2019-12-31
    lon             (y, x) float64 10MB ...
    lat             (y, x) float64 10MB ...
Dimensions without coordinates: nbounds
Data variables:
    lat_bnds        (y, x, nbounds) float64 41MB ...
    lon_bnds        (y, x, nbounds) float64 41MB ...
    tasmax          (time, y, x) float32 2GB ...
    LambertParisII  int32 4B ...
Attributes: (12/34)
    title:                         Anastasia-V1
    summary:                       Daily temperature dataset over metropolita...
    keywords:                      temperature, climate, France, surface air ...
    Conventions:                   CF-1.6, ACDD-1.3
    license:                       http://www.meteofrance.fr/licence
    history:                       File created on Fri Sep 12 20:52:51 2025
    ...                            ...
    publisher_email:               dcsc_ema@meteo.fr
    source:                        Spatialized surface air temperature observ...
    comment:                       Dataset produced by Meteo-France as part o...
    date_created:                  2025-09-12T20:52:51Z
    standard_name_vocabulary:      DCSC/EMA
    references:                    Besson, François & Dubuisson, Brigitte & E...

In [8]:
subset = tasmax_ds[['tasmax']].sel(time = slice("2019-06-01", "2019-09-30"))
subset = subset.to_dataframe()

In [20]:
df_ana = subset.reset_index().dropna()

In [17]:
df_gares = pd.read_csv("/Users/emile/Desktop/Datathon_MF/data/gares_map_projection.csv")

In [21]:
df_ana

,index,time,y,x,tasmax,lon,lat
0,14016,2019-06-01,1616000.0,1175000.0,295.841278,9.184015,41.326023
1,14017,2019-06-01,1616000.0,1176000.0,295.844452,9.195863,41.325241
2,14018,2019-06-01,1616000.0,1177000.0,295.865784,9.207711,41.324459
3,14022,2019-06-01,1616000.0,1181000.0,296.045319,9.255100,41.321320
4,15185,2019-06-01,1617000.0,1173000.0,295.844025,9.161349,41.336514
...,...,...,...,...,...,...,...
70550643,154428540,2019-09-30,2681000.0,613000.0,291.235413,2.521627,51.121326
70550644,154428541,2019-09-30,2681000.0,614000.0,291.243622,2.535870,51.121307
70550645,154428542,2019-09-30,2681000.0,615000.0,291.254181,2.550112,51.121284
70550646,154428543,2019-09-30,2681000.0,616000.0,291.263275,2.564355,51.121258


In [24]:
df_proj = df_ana.reset_index()
df_proj = df_proj[df_proj['time'] == "2019-06-01"]
df_proj.drop(['x', 'y', 'time', 'tasmax'], axis=1, inplace=True)
df_proj

,level_0,index,lon,lat
0,0,14016,9.184015,41.326023
1,1,14017,9.195863,41.325241
2,2,14018,9.207711,41.324459
3,3,14022,9.255100,41.321320
4,4,15185,9.161349,41.336514
...,...,...,...,...
578279,578279,1260569,2.521627,51.121326
578280,578280,1260570,2.535870,51.121307
578281,578281,1260571,2.550112,51.121284
578282,578282,1260572,2.564355,51.121258


In [31]:
def find_nearest_station(
    df_proj: pd.DataFrame, df_station: pd.DataFrame
) -> pd.DataFrame:
    # Calcule les distances à une gare
    distances = cdist(
        df_proj[["lat", "lon"]],
        df_station[["lat", "lon"]],
        metric="euclidean",  # lambda a, b: geodesic(a, b).kilometers,
    )
    # Détermine la station la plus proche
    df_proj[["gare"]] = df_station[["gare"]].to_numpy()[distances.argmin(axis=1)]
    return df_proj

In [ ]:
df_map = df_proj.copy()
df_map = find_nearest_station(df_proj=df_map, df_station=df_gares)
df_map

In [32]:
def find_nearest_station(
    df_gares: pd.DataFrame, df_proj: pd.DataFrame
) -> pd.DataFrame:
    # Calcule les distances à une gare
    distances = cdist(
        df_gares[["lat", "lon"]],
        df_proj[["lat", "lon"]],
        metric="euclidean",  # lambda a, b: geodesic(a, b).kilometers,
    )
    # Détermine la station la plus proche
    df_gares[["lat_proj", "lon_proj"]] = df_proj[["lat", "lon"]].to_numpy()[distances.argmin(axis=1)]
    return df_gares

In [33]:
df_map = df_proj.copy()
df_map = find_nearest_station(df_gares=df_gares, df_proj=df_proj)
df_map

,gare,lat,lon,lat_proj,lon_proj
0,Abancourt,49.685224,1.774306,49.684536,1.769002
1,Abbaretz,47.554643,-1.524416,47.554546,-1.518160
2,Abbeville,50.102210,1.824490,50.097778,1.820177
3,Ablon-sur-Seine,48.725468,2.419151,48.724487,2.418020
4,Achères Grand Cormier,48.955183,2.091903,48.957935,2.090862
...,...,...,...,...,...
2773,Yffiniac,48.470246,-2.652463,48.466305,-2.655128
2774,Ygos-Saint-Saturnin,43.978185,-0.736153,43.976944,-0.739284
2775,Ytrac,44.910689,2.364447,44.910702,2.361872
2776,Yvetot,49.622035,0.750115,49.621529,0.746619


In [34]:
df = pd.merge(
    left=df_ana, right=df_map, how='left', left_on=['lat', 'lon'], right_on=['lat_proj', 'lon_proj']
).dropna().drop(['lat_proj', 'lon_proj'], axis=1)
df.head()

,index,time,y,x,tasmax,lon_x,lat_x,gare,lat_y,lon_y
7026,128167,2019-06-01,1714000.0,568000.0,294.603088,1.948774,42.429394,Bourg-Madame,42.432407,1.948670
7549,130609,2019-06-01,1716000.0,668000.0,298.521057,3.160911,42.444866,Cerbère,42.441773,3.163403
7661,131676,2019-06-01,1717000.0,564000.0,294.920105,1.900096,42.456139,Latour-de-Carol - Enveitg,42.459046,1.904503
7672,131687,2019-06-01,1717000.0,575000.0,293.496765,2.033466,42.456604,Saillagouse,42.456885,2.032113
8408,135290,2019-06-01,1720000.0,665000.0,298.655212,3.125033,42.481052,Banyuls-sur-Mer,42.482958,3.125050


In [35]:
tasmin_file = "data/REA_ANASTASIA_raw/tasmin_France-Metro_ANASTASIA-V1_day_20190101_20191231.nc"
tasmin_ds = xr.open_dataset(f"../{tasmin_file}")

tasmin_ds

<xarray.Dataset> Size: 2GB
Dimensions:         (y: 1081, x: 1171, nbounds: 4, time: 365)
Coordinates:
  * y               (y) float64 9kB 1.605e+06 1.606e+06 ... 2.684e+06 2.685e+06
  * x               (x) float64 9kB 4e+04 4.1e+04 4.2e+04 ... 1.209e+06 1.21e+06
  * time            (time) datetime64[ns] 3kB 2019-01-01 ... 2019-12-31
    lon             (y, x) float64 10MB ...
    lat             (y, x) float64 10MB ...
Dimensions without coordinates: nbounds
Data variables:
    lat_bnds        (y, x, nbounds) float64 41MB ...
    lon_bnds        (y, x, nbounds) float64 41MB ...
    tasmin          (time, y, x) float32 2GB ...
    LambertParisII  int32 4B ...
Attributes: (12/34)
    title:                         Anastasia-V1
    summary:                       Daily temperature dataset over metropolita...
    keywords:                      temperature, climate, France, surface air ...
    Conventions:                   CF-1.6, ACDD-1.3
    license:                       http://www.meteofrance.fr/licence
    history:                       File created on Fri Sep 12 20:52:23 2025
    ...                            ...
    publisher_email:               dcsc_ema@meteo.fr
    source:                        Spatialized surface air temperature observ...
    comment:                       Dataset produced by Meteo-France as part o...
    date_created:                  2025-09-12T20:52:23Z
    standard_name_vocabulary:      DCSC/EMA
    references:                    Besson, François & Dubuisson, Brigitte & E...

In [36]:
subset = tasmin_ds[['tasmin']].sel(time = slice("2019-06-01", "2019-09-30"))
subset = subset.to_dataframe()

In [37]:
df_ana_tasmin = subset.reset_index().dropna()

In [38]:
df_ana_tasmin

,time,y,x,tasmin,lon,lat
14016,2019-06-01,1616000.0,1175000.0,288.930878,9.184015,41.326023
14017,2019-06-01,1616000.0,1176000.0,288.961395,9.195863,41.325241
14018,2019-06-01,1616000.0,1177000.0,288.974304,9.207711,41.324459
14022,2019-06-01,1616000.0,1181000.0,288.936157,9.255100,41.321320
15185,2019-06-01,1617000.0,1173000.0,288.762848,9.161349,41.336514
...,...,...,...,...,...,...
154428540,2019-09-30,2681000.0,613000.0,287.165588,2.521627,51.121326
154428541,2019-09-30,2681000.0,614000.0,287.135010,2.535870,51.121307
154428542,2019-09-30,2681000.0,615000.0,287.103973,2.550112,51.121284
154428543,2019-09-30,2681000.0,616000.0,287.074219,2.564355,51.121258


In [45]:
df_full = pd.merge(how='left',
    left=df,
    right=df_ana_tasmin,
    left_on=['lat_x', 'lon_x', 'time'],
    right_on=['lat', 'lon', 'time']
).dropna().drop(['x_y', 'y_y'], axis=1).rename(columns={'x_x': 'x', 'y_x': 'y'})
df_full

,index,time,y,x,tasmax,lon_x,lat_x,gare,lat_y,lon_y,tasmin,lon,lat
0,128167,2019-06-01,1714000.0,568000.0,294.603088,1.948774,42.429394,Bourg-Madame,42.432407,1.948670,283.434479,1.948774,42.429394
1,130609,2019-06-01,1716000.0,668000.0,298.521057,3.160911,42.444866,Cerbère,42.441773,3.163403,286.886108,3.160911,42.444866
2,131676,2019-06-01,1717000.0,564000.0,294.920105,1.900096,42.456139,Latour-de-Carol - Enveitg,42.459046,1.904503,283.181946,1.900096,42.456139
3,131687,2019-06-01,1717000.0,575000.0,293.496765,2.033466,42.456604,Saillagouse,42.456885,2.032113,282.589600,2.033466,42.456604
4,135290,2019-06-01,1720000.0,665000.0,298.655212,3.125033,42.481052,Banyuls-sur-Mer,42.482958,3.125050,286.901154,3.125033,42.481052
...,...,...,...,...,...,...,...,...,...,...,...,...,...
338911,154408626,2019-09-30,2664000.0,606000.0,291.125061,2.421668,50.969036,Bergues,50.969010,2.425740,286.042328,2.421668,50.969036
338912,154409776,2019-09-30,2665000.0,585000.0,291.582825,2.123449,50.977848,Gravelines,50.978736,2.123285,285.411041,2.123449,50.977848
338913,154413302,2019-09-30,2668000.0,598000.0,291.210449,2.308054,51.004925,Grande-Synthe,51.002688,2.313355,286.473297,2.308054,51.004925
338914,154414478,2019-09-30,2669000.0,603000.0,291.092987,2.379108,51.013885,Coudekerque-Branche,51.017707,2.375142,286.736633,2.379108,51.013885


In [46]:
df_full = df_full.drop(['lat_y', 'lon_y', 'lat_x', 'lon_x', 'x', 'y'], axis=1)

In [47]:
df_full['tasmax'] = df_full['tasmax'] - 273.15
df_full['tasmin'] = df_full['tasmin'] - 273.15
df_full

,index,time,tasmax,gare,tasmin,lon,lat
0,128167,2019-06-01,21.453094,Bourg-Madame,10.284485,1.948774,42.429394
1,130609,2019-06-01,25.371063,Cerbère,13.736115,3.160911,42.444866
2,131676,2019-06-01,21.770111,Latour-de-Carol - Enveitg,10.031952,1.900096,42.456139
3,131687,2019-06-01,20.346771,Saillagouse,9.439606,2.033466,42.456604
4,135290,2019-06-01,25.505219,Banyuls-sur-Mer,13.751160,3.125033,42.481052
...,...,...,...,...,...,...,...
338911,154408626,2019-09-30,17.975067,Bergues,12.892334,2.421668,50.969036
338912,154409776,2019-09-30,18.432831,Gravelines,12.261047,2.123449,50.977848
338913,154413302,2019-09-30,18.060455,Grande-Synthe,13.323303,2.308054,51.004925
338914,154414478,2019-09-30,17.942993,Coudekerque-Branche,13.586639,2.379108,51.013885


In [50]:
cond = df_full["tasmin"] > 20
df_full["two_consecutive_nights"] = (
    cond.groupby([df_full["lat"], df_full["lon"]]).shift(1)
    & cond.groupby([df_full["lat"], df_full["lon"]]).shift(2)
)
df_full_tasmin_true = df_full[df_full["two_consecutive_nights"] == True]
df_full_tasmin_true

,index,time,tasmax,gare,tasmin,lon,lat,two_consecutive_nights
53047,24354239,2019-06-20,25.241791,Saint-Laurent-du-Var,19.015289,7.192811,43.659847,True
53052,24355412,2019-06-20,25.088043,Nice Saint-Augustin,19.094971,7.218292,43.667706,True
61219,28095441,2019-06-23,29.098938,Marseille Blancarde,18.282928,5.400435,43.293705,True
61222,28096610,2019-06-23,28.890594,Marseille Saint-Charles,18.658081,5.376326,43.303383,True
61226,28097780,2019-06-23,29.248138,Arenc Euroméditerranée,18.662933,5.364505,43.312710,True
...,...,...,...,...,...,...,...,...
308623,140812531,2019-09-20,25.613617,Saint-Laurent-du-Var,19.041962,7.192811,43.659847,True
308628,140813704,2019-09-20,25.587433,Nice Saint-Augustin,19.047668,7.218292,43.667706,True
308658,140827777,2019-09-20,26.779572,Carnoles,19.456207,7.487490,43.763309,True
308659,140828948,2019-09-20,26.715790,Menton,19.425720,7.488303,43.772282,True


In [53]:
df_full_tasmin_true['lag_1'] = df_full_tasmin_true['tasmax'].shift(1)
df_full_tasmin_true['lag_2'] = df_full_tasmin_true['tasmax'].shift(2)

mask_cond = (
    (df_full_tasmin_true['tasmax'] > 40) &
    (df_full_tasmin_true['lag_1'] > 35) &
    (df_full_tasmin_true['lag_2'] > 35)
)
df_cond_day = df_full_tasmin_true[mask_cond]

df_cond_day = df_cond_day.drop(['lag_1', 'lag_2'], axis=1)

df_cond_day

/var/folders/7s/jpp97hld13z3q4sx8jmfs4gm0000gn/T/ipykernel_22023/1909706449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_tasmin_true['lag_1'] = df_full_tasmin_true['tasmax'].shift(1)
/var/folders/7s/jpp97hld13z3q4sx8jmfs4gm0000gn/T/ipykernel_22023/1909706449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_tasmin_true['lag_2'] = df_full_tasmin_true['tasmax'].shift(2)


,index,time,tasmax,gare,tasmin,lon,lat,two_consecutive_nights
75135,34435096,2019-06-28,40.840576,Sète,27.343475,3.692549,43.409660,True
75137,34436375,2019-06-28,40.882294,Martigues,23.103058,5.023512,43.393692,True
78165,35835430,2019-06-29,40.677094,Cahors,19.582916,1.432661,44.448292,True
78192,35851875,2019-06-29,40.501648,Capdenac,21.087189,2.072321,44.577606,True
78224,35868279,2019-06-29,40.099670,Maurs,19.623779,2.197837,44.703747,True
...,...,...,...,...,...,...,...,...
152736,69550988,2019-07-25,41.725098,Mont de Terre,22.941406,3.097841,50.616978,True
152741,69552163,2019-07-25,41.826904,Annappes,23.021851,3.154381,50.625587,True
152742,69552163,2019-07-25,41.826904,Ascq,23.021851,3.154381,50.625587,True
152743,69553328,2019-07-25,41.761810,Lille Europe,22.737915,3.069907,50.635086,True


In [ ]:
import pandas as pd

df_cond_day['date'] = pd.to_datetime(df_cond_day['time']).dt.date
df_counts = df_cond_day.groupby('date').size().reset_index(name='count')

fig = px.line(df_counts, x='date', y='count',
             labels={'date': 'Date', 'count': 'Number of entries'},
             title='Number of Entries per Day')
fig.show()



In [57]:
df_counts

,date,count
0,2019-06-28,2
1,2019-06-29,15
2,2019-07-23,2
3,2019-07-24,22
4,2019-07-25,58


In [71]:
full_dates = pd.date_range(start='2019-06-01', end='2019-09-30', freq='D')
full_dates

DatetimeIndex(['2019-06-01', '2019-06-02', '2019-06-03', '2019-06-04',
               '2019-06-05', '2019-06-06', '2019-06-07', '2019-06-08',
               '2019-06-09', '2019-06-10',
               ...
               '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24',
               '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28',
               '2019-09-29', '2019-09-30'],
              dtype='datetime64[ns]', length=122, freq='D')

In [72]:
count_dict = {}

In [73]:
for date in full_dates:
    if date.date() not in df_counts['date'].values:
        count_dict[date.date()] = 0
    else:
        count_value = df_counts[df_counts['date'] == date.date()]['count'].values[0]
        count_dict[date.date()] = count_value

In [75]:
count_df = pd.DataFrame(list(count_dict.items()), columns=['date', 'count'])
count_df

,date,count
0,2019-06-01,0
1,2019-06-02,0
2,2019-06-03,0
3,2019-06-04,0
4,2019-06-05,0
...,...,...
117,2019-09-26,0
118,2019-09-27,0
119,2019-09-28,0
120,2019-09-29,0


In [77]:
fig = px.line(count_df, x='date', y='count',
             labels={'date': 'Date', 'count': 'Nombre de gares touchées'},
             title="Nombre de gares vérifiant les conditions à l'été 2019")
fig.show()

In [79]:
date = "2019-07-25"
df_to_plot = df_cond_day[df_cond_day['time'] == date]
df_to_plot = df_to_plot.sort_values('tasmax')
df_to_plot.shape

(58, 9)

In [80]:
fig = px.scatter_map(data_frame=df_to_plot, lat='lat', lon='lon',
                     color="tasmax", size="tasmax", range_color=[40, 46],
                     color_continuous_scale=["white", "yellow", "orange", "purple"],
                     size_max=10,
                     zoom=4.5, height=700, width=900,
                     title="Gares vérifiant les conditions le 25 juillet 2019"
)
fig.show()